##### Router Query Engine

In [ ]:
!pip install llama_index==0.9.31

In [1]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [2]:
import logging
import sys
import os
from dotenv import load_dotenv

load_dotenv(".env", override=True)
openai_key = os.environ["OPENAI_API_KEY"] 

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    ListIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
)

In [3]:
# load documents
documents = SimpleDirectoryReader("data").load_data()

# initialize service context (set chunk size)
service_context = ServiceContext.from_defaults(chunk_size=1024)
nodes = service_context.node_parser.get_nodes_from_documents(documents)

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

##### Define List Index and Vector Index over Same Data

In [4]:
list_index = ListIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)


HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


##### Define Query Engines and Set Metadata

In [5]:
list_query_engine = list_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [6]:
from llama_index.tools.query_engine import QueryEngineTool


list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description="Useful for summarization questions related to Paul Graham eassy on What I Worked On.",
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context from Paul Graham essay on What I Worked On.",
)

##### Define Router Query Engine

In [7]:
from llama_index.query_engine.router_query_engine import RouterQueryEngine

from llama_index.selectors.pydantic_selectors import (
    PydanticSingleSelector
)


query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

In [8]:
response = query_engine.query("What is the summary of the document?")
print("=========")
response.response

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Selecting query engine 0: This choice is specifically mentioned as useful for summarization questions..
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


"The document provides a detailed account of an individual's journey through various fields, including writing, programming, art, and technology. It follows the narrative of the author's exploration of different interests, experiences in art school, work in the tech industry, and entrepreneurial endeavors. The story highlights the individual's transitions between painting, programming, and investing, showcasing a diverse and impactful career trajectory. It also delves into the development of the Lisp programming language, the creation of new Lisp dialects, and the challenges faced during these projects. Personal reflections on experiences, decisions, and career shifts are woven throughout the narrative, offering insights into the individual's evolving interests and ventures."

In [9]:
response = query_engine.query("What did Paul Graham do after RICS?")
print("=========")
response.response

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Selecting query engine 1: The question is asking for specific context from Paul Graham's essay on What I Worked On..
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'After leaving Y Combinator, Paul Graham decided to focus on painting.'